In [52]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd
import requests
from newspaper import Article
import numpy as np
from collections import defaultdict

In [53]:
def sentimentAnalysis(text, dictionary, url):
    if(len(dictionary) == 0):
        dictionary = {
            "URL": [],
            "Sentiment Score": [],
            "Sentiment Label": [],
            "Subjectivity Score": [],
            "Positive Words": [],
            "Negative Words": [],
            "Text": []
            }
    if(text[0:8] != "PARERROR"):
        # Start the sentiment analysis now
        dictionary["URL"].append(url)
        doc = nlp(text)
        sentiment = doc._.blob.polarity
        sentiment = round(sentiment,2)
        subjectivity = doc._.blob.subjectivity
        subjectivity = round(subjectivity,2)

        # Gives positive or negative label
        if sentiment >= 0.033 and sentiment <= 0.043:
            sent_label = "Neutral"
        elif sentiment > 0.043 and sentiment < 0.143:
            sent_label = "Neutral Positive"
        elif sentiment > 0.143:
            sent_label = "Positive"
        elif sentiment < 0.033 and sentiment > -0.062:
            sent_label = "Neutral Negative"
        elif sentiment < -0.062:
            sent_label = "Negative"

        dictionary["Sentiment Label"].append(sent_label)
        dictionary["Sentiment Score"].append(sentiment)
        dictionary["Subjectivity Score"].append(subjectivity)
        dictionary["Text"].append(text)

        positive_words = []
        negative_words = []

        for x in doc._.blob.sentiment_assessments.assessments:
          if x[1] > 0:
            positive_words.append(x[0][0])
          elif x[1] < 0:
            negative_words.append(x[0][0])
          else:
            pass

        dictionary["Positive Words"].append(', '.join(set(positive_words)))
        dictionary["Negative Words"].append(', '.join(set(negative_words)))
    
    else:
        dictionary["URL"].append(url)
        dictionary["Sentiment Label"].append(text)
        dictionary["Sentiment Score"].append(0.0)
        dictionary["Subjectivity Score"].append(0.0)
        dictionary["Text"].append(text)

        positive_words = []
        negative_words = []

        dictionary["Positive Words"].append(', '.join(set(positive_words)))
        dictionary["Negative Words"].append(', '.join(set(negative_words)))
    
    return dictionary

# Topic Level Sentiment

In [54]:
# def get_sentences(doc):
#     return doc.sents

# def get_sentence_list(doc):
#     return [sent for sent in doc.sents]

# #Takes a doc object from spacy and returns a tuple list of form (sentence, sentiment of sentence) for all sentences
# def sentence_sentiment_from_doc(doc):
#     sentences = get_sentences(doc)
#     tuple_list = []
#     for sentence in sentences:
#         sent_doc = nlp(sentence.text)
#         tuple_list.append((sentence.text,sent_doc._.blob.polarity))
#     return tuple_list

# #takes in a single topic word, the word's weight, and the doc, and returns sentiment of that word within the doc
# def sentence_level_sentiment_of_word(word, weight, doc):
#     sentence_sentiment_list = sentence_sentiment_from_doc(doc)
#     sentiment_total = 0
    
#     for (sentence, sentiment) in sentence_sentiment_list:
#              if sentence.find(word) != -1:
#                     #print(name,sentence)
#                     sentiment_total += sentiment
#     return sentiment_total

# def topic_level_sentiment(ldamodel):
#     my_dict = {'Topic_' + str(i): [token for token, score in ldamodel.show_topic(i, topn=10)] for i in range(0, ldamodel.num_topics)}
    
#     return my_dict

#returns a dictionary of all topics, with all their associated topic words in the form {Topic: [words]}
def create_topic_words_dict(ldamodel):
    my_dict = {'Topic_' + str(i): [token for token, score in ldamodel.show_topic(i, topn=10)] for i in range(0, ldamodel.num_topics)}
    
    return my_dict

#returns all sentences in a document as a list
def get_sentences(doc):
    return doc.sents

#Takes a doc object from spacy and returns a tuple list of form (sentence, sentiment of sentence) for all sentences
def sentence_sentiment_from_doc(doc):
    sentences = get_sentences(doc)
    tuple_list = []
    for sentence in sentences:
        sent_doc = nlp(sentence.text)
        tuple_list.append((sentence.text,sent_doc._.blob.polarity)) #list of tuples of form [(text, sentiment)]
    return tuple_list

#Returns an average sentiment score of all topics for a single document
def sentence_sentiment_on_topics(doc, topic_list, df_topics):
    sentence_sentiment_list = sentence_sentiment_from_doc(doc) #get all sentences and their sentiment
    score_list = []
    return_dict = {}
    
    for key in topic_list: #for every topic
        for topic in df_topics: # every topic within our current article
            print(key[-1])
            print(topic[0])
            print(type(key[-1]))
            print(type(topic[0]))
            if int(key[-1]) == topic[0]: # Only does sentiment on topics that are part of the related topics
                print("HERE")
                for word in topic_list[key]: #for every word in that topic
                    for sentence, sentiment in sentence_sentiment_list:
                         if sentence.find(word) != -1: #if the word is in that sentence we add the sentiment value
                                score_list.append(sentiment)
                if not score_list:
                    return_dict[key] = 0
                else:
                    return_dict[key] = sum(score_list) / len(score_list) #average of all sentence sentiments for topic
    
    return return_dict

def topic_sentence_sentiment_analysis(df, LDA_model, corpus):
    #cleaneddf = drop_failed_webscraping_rows(df)
    #LDA_model, corpus = create_lda_model(cleaneddf, 20, 5, 5)

    topicSentDic = {}
    for x in range(len(df["URL"])): #for every article
        page_text = df.iloc[x]["Text"]
        df_topics = df.iloc[x]["Topics"]
        tempdoc = nlp(page_text) #gather page text and transform into doc object
        topic_list = create_topic_words_dict(LDA_model) #list of topics and their words
        temp = sentence_sentiment_on_topics(tempdoc,topic_list, df_topics) #dictionary of all topics and their average sentiment for the article
        topicSentDic[df.iloc[x]["URL"]] = temp #append sentiment dict
    
    return topicSentDic

In [ ]:
# Pass in a word and a document and get back the average sentiment of that word for the document.
def word_sentiment_per_doc(word, sentence_sentiment_list):
    word_score = 0
    total_appearences = 0
    for sentence, sentiment in sentence_sentiment_list:
        if sentence.find(word) != -1: #if the word is in that sentence we add the sentiment value
            word_score += sentiment 
            total_appearences += 1
    if total_appearences == 0:
        return None
    word_sentiment = word_score / total_appearences
    return word_sentiment

def topic_sentiment_per_doc(topic_words, text):
    weighted_topic_sentiment = 0
    for word, score in topic_words:
        # Get list of sentences whithin the document
        doc = nlp(text)
        sentence_sentiment_list = sentence_sentiment_from_doc(doc) # get all sentences and their sentiment
        # For each word in a topic
        # Multiply the relavence by the sentiment to get a weighted sentiment
        word_sentiment = word_sentiment_per_doc(word, sentence_sentiment_list)
        if word_sentiment != None:
            weighted_word_sentiment = score * word_sentiment
            weighted_topic_sentiment += weighted_word_sentiment
    return weighted_topic_sentiment

# The return value is a {Topic: [(document, sentiment), (document, sentiment), ...], Topic: [(document, sentiment), (document, sentiment), ...], ...}

# Sentiment per topic per document
def topics_per_article(corpus, ldamodel):
    topics_list = defaultdict(list)
    for doc_id, doc_bow in enumerate(corpus):
        doc_topics = ldamodel.get_document_topics(doc_bow)
        for topic_id, score in doc_topics:
            topics_list[doc_id].append((topic_id, score))
    return topics_list

def add_sentiment(df, LDA_model, rawData):
    # Create a new DataFrame to store the updated values
    data = []
    
    for document_id in df.index:
        row = []
        for topic_id in df.columns:
            topic_words = LDA_model.show_topic(topic_id[0])
            current_value = df.at[document_id, topic_id]
            new_value = [current_value, topic_sentiment_per_doc(topic_words, rawData[document_id])]
            row.append(new_value)
            print(((int(topic_id[0]) / df.shape[1]) + int(document_id)) / df.shape[0])
        data.append(row)

    new_df = pd.DataFrame(data, columns=[i for i in range(df.shape[1])])

    return new_df